In [ ]:
!pip install -q feedparser 2>/dev/null

import pandas as pd
import urllib.parse, feedparser
from datetime import datetime, timedelta
from google.colab import ai
from IPython.display import Markdown, display
import time
import json
import re

# === CONFIG ===
topics = [
    "NVDA OR NVIDIA", "AAPL OR Apple", "TSLA OR Tesla", "AMD",
    "META OR Facebook", "MSFT OR Microsoft", "GOOGL OR Google OR Alphabet",
    "AMZN OR Amazon", "JPMorgan OR JPM", "COIN OR Coinbase"
]
days_back = 7

# === Beautiful box function (same as FinViz) ===
def print_sentiment_box(ticker, pos, neg, neu, net_score, signal, tone):
    if "BUY" in signal:
        color = "\033[92m"  # Green
        arrow = "Green Up Arrow"
    elif "SELL" in signal:
        color = "\033[91m"  # Red
        arrow = "Red Down Arrow"
    else:
        color = "\033[93m"  # Yellow
        arrow = "Yellow Circle"
    reset = "\033[0m"
    print("┌" + "─" * 58 + "┐")
    print("│" + " GOOGLE NEWS + GEMINI SENTIMENT ".center(58) + "│")
    print("│" + f" {ticker.upper()} ".center(58) + "│")
    print("├" + "─" * 58 + "┤")
    print(f"│  Positive   : {pos:5.1f}%{' ':>37}│")
    print(f"│  Negative   : {neg:5.1f}%{' ':>37}│")
    print(f"│  Neutral    : {neu:5.1f}%{' ':>37}│")
    print("├" + "─" * 58 + "┤")
    print(f"│  Net Sentiment Score : {net_score:+.4f}{' ':>27}│")
    print("├" + "─" * 58 + "┤")
    print("│" + " " * 58 + "│")
    print("│" + " RECOMMENDED ACTION: ".center(58) + "│")
    print("│" + " " * 58 + "│")
    print("│" + f"   {color}▶▶ {signal.upper():^32} ◀◀{reset}".center(58) + "       │")
    print(f"│   {arrow}  Market Tone: {tone:^22}    │")
    print("│" + " " * 58 + "│")
    print("└" + "─" * 58 + "┘\n")

# === Main loop ===
results = []
today = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")

print(f"Fetching latest {days_back}-day Google News for 10 stocks/topics using Gemini...\n")

for idx, query in enumerate(topics, 1):
    print(f"({idx:2}/10) Analyzing → {query.split()[0]:<6}", end=" \n")

    url = f"https://news.google.com/rss/search?q={urllib.parse.quote(query)}+after:{start_date}"
    feed = feedparser.parse(url)
    headlines = [e.title for e in feed.entries[:100]]  # Top 100 recent

    if len(headlines) < 5:
        print("Too few articles")
        continue

    context = " ".join(headlines)

    prompt = f"""Analyze the sentiment of these financial news headlines about {query.split()[0]}.
Return ONLY a valid JSON object like this (no extra text, no markdown):

{{
  "positive_percent": 65.4,
  "negative_percent": 12.3,
  "neutral_percent": 22.3,
  "net_score": 0.531,
  "overall_tone": "Strongly Bullish"
}}

Headlines:
{context[:12000]}"""

    try:
        # FIXED: No 'temperature' parameter!
        raw = ai.generate_text(prompt)

        # Better JSON extraction (handles if Gemini wraps in ```json)
        json_match = re.search(r'\{.*\}', raw, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
        else:
            json_str = raw.strip()

        data = json.loads(json_str)

        pos = data.get("positive_percent", 0)
        neg = data.get("negative_percent", 0)
        neu = data.get("neutral_percent", 100 - pos - neg)
        net = round((pos - neg) / 100, 4)
        tone = data.get("overall_tone", "Neutral")

        signal = "BUY (Long)" if pos > 40 else "SELL SHORT" if neg > 40 else "NO ACTION"

        print_sentiment_box(query.split()[0], pos, neg, neu, net, signal, tone)

        results.append({
            "Date": today,
            "Ticker": query.split()[0],
            "Articles": len(headlines),
            "Positive_%": round(pos, 1),
            "Negative_%": round(neg, 1),
            "Neutral_%": round(neu, 1),
            "Net_Score": net,
            "Signal": signal.split("(")[0].strip(),
            "Tone": tone
        })

        print(f"✓ Success! (Parsed {len(headlines)} articles)")

    except json.JSONDecodeError as e:
        print(f"JSON parse error → {e}")
        print(f"Raw response: {raw[:200]}...")
    except Exception as e:
        print(f"Error → {e}")

    time.sleep(2)  # Rate limit safety

# === Save CSV ===
if results:
    df_final = pd.DataFrame(results)
    csv_file = f"GoogleNews_Gemini_Sentiment_10Stocks_{today}.csv"
    df_final.to_csv(csv_file, index=False)

    # === Final summary table ===
    print("═" * 80)
    print(" " * 28 + "FINAL GEMINI SENTIMENT RESULTS")
    print("═" * 80)
    display(df_final.style
            .background_gradient(subset=['Net_Score'], cmap='RdYlGn')
            .format({"Positive_%": "{:.1f}%", "Negative_%": "{:.1f}%", "Neutral_%": "{:.1f}%"}))

    print(f"\nCSV exported → {csv_file}")
    print(f"Total articles scanned: {df_final['Articles'].sum():,}")
else:
    print("No results to save. Check errors above.")

Fetching latest 7-day Google News for 10 stocks/topics using Gemini...

( 1/10) Analyzing → NVDA   
┌──────────────────────────────────────────────────────────┐
│              GOOGLE NEWS + GEMINI SENTIMENT              │
│                           NVDA                           │
├──────────────────────────────────────────────────────────┤
│  Positive   :  23.0%                                     │
│  Negative   :  56.0%                                     │
│  Neutral    :  21.0%                                     │
├──────────────────────────────────────────────────────────┤
│  Net Sentiment Score : -0.3300                           │
├──────────────────────────────────────────────────────────┤
│                                                          │
│                   RECOMMENDED ACTION:                    │
│                                                          │
│       ▶▶            SELL SHORT            ◀◀           │
│   Red Down Arrow  Market Tone:        Bearish 